### Hybrid Search Langchain

In [2]:
import getpass
import os

In [11]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_TRACING_V2=True
PINE_CONE_API_KEY=os.getenv("PINE_CONE_API_KEY")
HF_TOKEN=os.getenv("HF_TOKEN")

In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [7]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=PINE_CONE_API_KEY)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [ ]:
index=pc.Index(index_name)
index

In [ ]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

#os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer


embeddings=HuggingFaceEmbeddings()
embeddings

In [ ]:
!pip install pinecone_text

In [ ]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [ ]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


In [11]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [ ]:
retriever

In [ ]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]
)

In [ ]:
retriever.invoke("What city did i visit first")